In [139]:
import casadi as ca
from cyecca import lie
import math
from cyecca.symbolic import taylor_series_near_zero
import sympy
import numpy as np


In [140]:
def derive_Ul_inv_closed_form():
    x = ca.SX.sym('x', 9)
    v = x[:3]
    a = x[3:6]
    r = x[6:]
    theta = ca.norm_2(r)

    A = lie.so3.elem(a).ad()
    V = lie.so3.elem(v).ad()
    Omega = lie.so3.elem(r).ad()
    Omega_sq = Omega @ Omega
    
    th = sympy.symbols('th')
    sin = sympy.sin
    cos = sympy.cos
    
    c_theta = ca.cos(theta)
    s_theta = ca.sin(theta)
    
    Coeff = ca.if_else(ca.fabs(theta) > 1e-3, 
        ca.vertcat(
            (1 - c_theta)/theta**2,
            (theta - s_theta)/theta**3,
            (2 - 2*c_theta - theta*s_theta)/(2*theta**4),
            (theta**2 + 2*c_theta - 2)/(2*theta**4),
            (theta*c_theta + 2*theta - 3*s_theta)/(2*theta**5),
            (theta**2 + theta*s_theta + 4*c_theta - 4)/(2*theta**6)),
        ca.vertcat(
            1/2 - theta**2/24 + theta**4/720,
            1/6 - theta**2/120 + theta**4/5040,
            1/24 - theta**2/360 + theta**4/13440,
            1/24 - theta**2/720 + theta**4/40320,
            1/120 - theta**2/2520 + theta**4/120960,
            1/720 - theta**2/210600 + theta**4/1209600)
        )
    
    R = ca.SX.eye(3) + Coeff[0] * Omega + Coeff[1] * Omega_sq
    
    C = A/2
    C += Coeff[1] * (Omega@A + A@Omega)
    C += Coeff[2] * (Omega@A@Omega)
    C += Coeff[3] * (Omega_sq@A + A@Omega_sq)
    C += Coeff[4] * (Omega_sq@A@Omega + Omega@A@Omega_sq)
    C += Coeff[5] * (Omega_sq@A@Omega_sq)
    
    f_C = ca.Function('f_C', [r, a], [C])
    
    C_A = f_C(r, a)
    C_V = f_C(r, v)

    Z = ca.SX.zeros(3, 3)
    Ul_inv = ca.sparsify(ca.vertcat(
        ca.horzcat(R, Z, C_A),
        ca.horzcat(Z, R, C_V),
        ca.horzcat(Z, Z, R)))
    return ca.Function('Ul_inv_closed_form', [x], [Ul_inv], ['x'], ['Ul_inv'])

Ul_inv_cf = derive_Ul_inv_closed_form()

In [142]:
def derive_Ul_inv_series(n):
    x = lie.se23.elem(ca.SX.sym('x', 9))
    X = x.ad()
    Ul_inv = ca.SX.zeros(9, 9)
    #print(ca.mpower(X,  0))
    for k in range(n):
        Xp = ca.mpower(X,  k)
        #print(k)
        #print(Xp.sparsity().spy())
        Ul_inv += Xp/math.factorial(k+1)
    Ul_inv = ca.sparsify(Ul_inv)
    return ca.Function('Ul_inv_series', [x.param], [Ul_inv])

Ul_inv_series = derive_Ul_inv_series(10)

In [155]:
x = lie.se23.elem(ca.DM([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.5, 0.6, 0.7]))

In [156]:
%%timeit
Ul_inv_cf(x.param)

158 µs ± 219 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [157]:
%%timeit
Ul_inv_series(x.param)

861 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [158]:
Ul_inv = np.array(ca.DM(Ul_inv_cf(x.param)))
Ul_inv

array([[ 0.86592, -0.27175,  0.3287 ,  0.     ,  0.     ,  0.     , -0.21477, -0.15072,  0.27158],
       [ 0.36639,  0.88328, -0.16166,  0.     ,  0.     ,  0.     ,  0.29657, -0.18483, -0.04084],
       [-0.21828,  0.29416,  0.90378,  0.     ,  0.     ,  0.     , -0.09879,  0.2526 , -0.14886],
       [ 0.     ,  0.     ,  0.     ,  0.86592, -0.27175,  0.3287 , -0.099  , -0.09272,  0.10611],
       [ 0.     ,  0.     ,  0.     ,  0.36639,  0.88328, -0.16166,  0.13959, -0.07758,  0.01708],
       [ 0.     ,  0.     ,  0.     , -0.21828,  0.29416,  0.90378, -0.04092,  0.07883, -0.04996],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.86592, -0.27175,  0.3287 ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.36639,  0.88328, -0.16166],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     , -0.21828,  0.29416,  0.90378]])

In [159]:
ca.inv(Ul_inv[:3,:3]) @ Ul_inv[:3,:3]

DM(
[[1, 0, -2.77556e-17], 
 [-2.08167e-17, 1, 0], 
 [-5.55112e-17, 5.55112e-17, 1]])

In [160]:
Ul_inv[:3,:3].T @ Ul_inv[:3,:3]

array([[0.93171, 0.0241 , 0.02812],
       [0.0241 , 0.94055, 0.03374],
       [0.02812, 0.03374, 0.95099]])

In [134]:
np.array(ca.DM(Ul_inv_series(x.param)))

array([[ 0.98, -0.14,  0.1 ,  0.  ,  0.  ,  0.  , -0.09, -0.27,  0.27],
       [ 0.15,  0.98, -0.04,  0.  ,  0.  ,  0.  ,  0.31, -0.07, -0.15],
       [-0.09,  0.06,  0.99,  0.  ,  0.  ,  0.  , -0.21,  0.24, -0.05],
       [ 0.  ,  0.  ,  0.  ,  0.98, -0.14,  0.1 , -0.04, -0.14,  0.11],
       [ 0.  ,  0.  ,  0.  ,  0.15,  0.98, -0.04,  0.15, -0.03, -0.03],
       [ 0.  ,  0.  ,  0.  , -0.09,  0.06,  0.99, -0.09,  0.07, -0.02],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.98, -0.14,  0.1 ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.15,  0.98, -0.04],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , -0.09,  0.06,  0.99]])

In [138]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(Ul_inv_cf(x.param)) - ca.DM(Ul_inv_series(x.param)))

array([[-1.24989e-12,  2.25903e-13,  2.66065e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00, -2.82378e-11,  5.60102e-12,  6.43652e-12],
       [ 1.58734e-13, -9.61453e-13,  5.88092e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  3.93069e-12, -2.18101e-11,  1.34444e-11],
       [ 3.10835e-13,  5.65707e-13, -4.80838e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  7.57250e-12,  1.28431e-11, -1.14820e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24989e-12,  2.25903e-13,  2.66065e-13, -1.24983e-11,  2.29233e-12,  2.63782e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.58734e-13, -9.61453e-13,  5.88092e-13,  1.55320e-12, -9.61405e-12,  5.89162e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  3.10835e-13,  5.65707e-13, -4.80838e-13,  3.13059e-12,  5.64523e-12, -4.80702e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24989e-12,  2.25903e-13,  2.66065e-13],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00